In [1]:
# import libraries - modin needed for http
import modin.pandas as pd
import pytz

In [ ]:
# install timezone
%pip install pytz

In [3]:
# read the http file into the test dataframe
df_test = pd.read_csv('http.csv')

(raylet) Spilled 2640 MiB, 6 objects, write throughput 920 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


In [4]:
# check - number of url series
len(df_test['url'])

23555143

(raylet) Spilled 4398 MiB, 10 objects, write throughput 489 MiB/s.


In [5]:
# check - number of unique url
df_test['url'].nunique()

(raylet) Spilled 8355 MiB, 19 objects, write throughput 466 MiB/s.
(raylet) Spilled 21111 MiB, 51 objects, write throughput 554 MiB/s.


6697

In [6]:
# break the domain out of the url for evaluation - story to new column
df_test['domain'] = df_test['url'].apply(lambda x: x.split('/')[2])

In [7]:
# number of unique domains
len(df_test['domain'].unique())

454

In [ ]:
# import libraries needed for api call
key = 'masked' # demo key anyway
url = 'https://investigate.api.umbrella.com'
import requests
import time

In [ ]:
# define function to pull domain categorization
def get_cat(key, url, domain):
    header = {'Authorization':'Bearer '+key,
        'Content-Type':'application/json'}
    
    response = requests.get(url+'/domains/categorization/'+domain+'?showLabels', headers=header).json()

    time.sleep(5)

    return response

In [ ]:
# check - function
get_cat(key, url, 'www.google.comtest')

In [ ]:
# create a dictionary of domains to store their categorization
# print response if error
domainCats = {}

for domain in df_test['domain'].unique():
   try:
      response = get_cat(key, url, domain)
      domainCats[domain] = response[domain]['content_categories']
   except:
      print(response)

In [8]:
# store the domain dictionary as a pickle in the fs
import pickle

#with open('domainCats.pkl', 'wb') as handle:
#    pickle.dump(domainCats, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('domainCats.pkl', 'rb') as handle:
    domainCats = pickle.load(handle)


In [9]:
# check - lenth of dictionary
len(domainCats.keys())

454

In [10]:
# check - validations that they match
len(df_test['domain'].unique())

454

In [11]:
# convert date to datetime in dataframe
df_test['date'] = pd.to_datetime(df_test['date'])

In [12]:
# dayDate converstion from date
df_test['dayDate'] = df_test['date'].dt.date

In [13]:
# date as string from date
df_test['dateString'] = df_test['date'].dt.strftime('%Y%m%d')

In [14]:
# generate a userDate field
df_test['userDate'] = df_test.apply(lambda x: x['user'] + '_' + x['dateString'], axis=1)

In [15]:
# number of rows in http test dataframe
len(df_test['id'])

23555143

In [16]:
# groupby userDaye and could the number of url. convert to dataframe and rename columns
df_urlByUserDate = df_test.groupby('userDate').count()['url'].to_frame().rename(columns={'url':'count'})

(raylet) Error processing line 1 of /home/vzb403/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "/usr/lib/python3.10/site.py", line 192, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "/home/vzb403/.local/lib/python3.10/site-packages/pandas/__init__.py", line 16, in <module>
(raylet)       raise ImportError(
(raylet)   ImportError: Unable to import required dependencies:
(raylet)   pytz: No module named 'pytz'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 1 of /home/vzb403/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet) Error processing line 1 of /home/vzb403/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "/usr/lib/python3.10/site.py", line 192, in addpackage
(raylet)       exec(l

In [17]:
# 338213 < 516000 = some userDates have no web activity, but some userDates have no login as well
len(df_urlByUserDate['count'])

338213

In [18]:
# save the userDate url to pickle
df_urlByUserDate.to_pickle('urlByUserDate.pkl')

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [19]:
# check - head of http test
df_test.head()

,id,date,user,pc,url,content,domain,dayDate,dateString,userDate
0,{U0G4-P5TW04EO-9366IUQF},2010-01-02 06:36:20,EBH0519,PC-9573,http://usbank.com/1930_FIFA_World_Cup/patenaud...,neros printing pastiche four topiary body pink...,usbank.com,2010-01-02,20100102,EBH0519_20100102
1,{G0R5-U4RX40DR-3710WPUY},2010-01-02 06:39:22,EBH0519,PC-9573,http://forbes.com/Chorioactis/urnula.html,balhae fubing guotu namsaeng geng chuzhou ying...,forbes.com,2010-01-02,20100102,EBH0519_20100102
2,{G0Q6-F5CV96RX-0355APNP},2010-01-02 06:39:53,EBH0519,PC-9573,http://coupons.com/BeauchampSharp_Tragedy/darb...,ziyi hazelnuts 34850 stat_area1 namsaeng kucha...,coupons.com,2010-01-02,20100102,EBH0519_20100102
3,{J8X0-C9ZO08VS-9370QIFO},2010-01-02 06:40:56,EBH0519,PC-9573,http://youtube.com/Amanita_ocreata/velosa.html,flambard anselm anselm precentor thurgot visit...,youtube.com,2010-01-02,20100102,EBH0519_20100102
4,{E0J4-D6WM72LS-9186JPKE},2010-01-02 06:47:23,EBH0519,PC-9573,http://twitter.com/Cannon/m198.html,maternus henrici anselm walkelin maternus thur...,twitter.com,2010-01-02,20100102,EBH0519_20100102


In [20]:
# set the categories of the domain in the dataframe based on the domainCate dictionary
df_test['domainCats'] = df_test.apply(lambda x: domainCats[x['domain']], axis=1)

(raylet) [2023-03-26 19:33:39,008 E 774348 774348] (raylet) node_manager.cc:3097: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 6809ec1799f2218e377e09d93bca1b02d67260156df5bfa672826b4c, IP: 192.168.0.202) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.0.202`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


In [21]:
# check - find all the values of categories
import itertools
set(itertools.chain.from_iterable(domainCats.values()))

{'Adult',
 'Adult Themes',
 'Advertisements',
 'Adware',
 'Arts',
 'Auctions',
 'Automotive',
 'Blogs',
 'Business Services',
 'Business and Industry',
 'Chat',
 'Classifieds',
 'Cloud and Data Centers',
 'Computer Security',
 'Computers and Internet',
 'DIY Projects',
 'Dating',
 'Digital Postcards',
 'Dining and Drinking',
 'Ecommerce/Shopping',
 'Education',
 'Educational Institutions',
 'Entertainment',
 'File Storage',
 'File Transfer Services',
 'Finance',
 'Financial Institutions',
 'Forums/Message boards',
 'Freeware and Shareware',
 'Games',
 'Government',
 'Government and Law',
 'Health and Fitness',
 'Health and Medicine',
 'Humor',
 'Illegal Downloads',
 'Infrastructure',
 'Instant Messaging',
 'Internet Telephony',
 'Job Search',
 'Jobs/Employment',
 'Lingerie and Swimsuits',
 'Lingerie/Bikini',
 'Lotteries',
 'Military',
 'Movies',
 'Music',
 'News',
 'News/Media',
 'Non-Profits',
 'Nudity',
 'Online Communities',
 'Online Document Sharing and Collaboration',
 'Online Mee

In [24]:
# set a list of categories that might be involved with insider threats based on scenarios - draft
insiderCats = ['File Storage',
 'File Transfer Services',
 'Forums/Message boards',
 'Freeware and Shareware',
 'Illegal Downloads',
 'Instant Messaging',
 'Internet Telephony',
 'Job Search',
 'Jobs/Employment',
 'Lingerie and Swimsuits',
 'Lingerie/Bikini',
 'Nudity',
 'Online Document Sharing and Collaboration',
 'Online Storage and Backup',
 'Photo Sharing',
 'Pornography',
 'Professional Networking',
 'Social Networking',
 'Video Sharing',
 'Web Spam',
 'Web-based Email',
 'Webmail']

In [25]:
# set a list of categories that might be involved with insider threats based on scenarios - draft
insiderCats = ['File Storage',
 'File Transfer Services',
 'Forums/Message boards',
 'Internet Telephony',
 'Job Search',
 'Jobs/Employment',
 'Online Document Sharing and Collaboration',
 'Online Storage and Backup',
 'Professional Networking',
 'Web-based Email',
 'Webmail']

In [26]:
# define a function to set insider threat value of 1 for each http row in http
def isInsiderWeb(row):
    if any(item in row['domainCats'] for item in insiderCats):
        return 1
    else:
        return 0

df_test['isInsiderWeb'] = df_test.apply(isInsiderWeb, axis=1)

(raylet) Error processing line 1 of /home/vzb403/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "/usr/lib/python3.10/site.py", line 192, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "/home/vzb403/.local/lib/python3.10/site-packages/pandas/__init__.py", line 16, in <module>
(raylet)       raise ImportError(
(raylet)   ImportError: Unable to import required dependencies:
(raylet)   pytz: No module named 'pytz'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 1 of /home/vzb403/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "/usr/lib/python3.10/site.py", line 192, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "/home/vzb403/.local/lib/python3.10/site-packages/p

In [27]:

df_test['domainCatsString'] = df_test.apply(lambda x: str(x['domainCats']), axis=1)

(raylet) Error processing line 1 of /home/vzb403/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "/usr/lib/python3.10/site.py", line 192, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "/home/vzb403/.local/lib/python3.10/site-packages/pandas/__init__.py", line 16, in <module>
(raylet)       raise ImportError(
(raylet)   ImportError: Unable to import required dependencies:
(raylet)   pytz: No module named 'pytz'
(raylet) 
(raylet) Remainder of file ignored
(raylet) Error processing line 1 of /home/vzb403/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:
(raylet) 
(raylet)   Traceback (most recent call last):
(raylet)     File "/usr/lib/python3.10/site.py", line 192, in addpackage
(raylet)       exec(line)
(raylet)     File "<string>", line 1, in <module>
(raylet)     File "/home/vzb403/.local/lib/python3.10/site-packages/p

In [28]:
# set the categories of the domain in the dataframe based on the domainCate dictionary - sum the quantity of the items per userDate
df_insiderWebCount = df_test[df_test['isInsiderWeb'] == 1].groupby('userDate').sum()

(raylet) [2023-03-26 19:38:39,013 E 774348 774348] (raylet) node_manager.cc:3097: 5 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 6809ec1799f2218e377e09d93bca1b02d67260156df5bfa672826b4c, IP: 192.168.0.202) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.0.202`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) Error processing line 1 of /home/vzb403/.local/lib/python3.10/site-packages/modin-autoimport-pandas.pth:

In [29]:
# check - new df for sum insiderthreat count
df_insiderWebCount

,isInsiderWeb
userDate,
AAC0070_20100104,1
AAC0070_20100105,10
AAC0070_20100106,4
AAC0070_20100107,3
AAC0070_20100111,2
...,...
ZSP0972_20110506,5
ZSP0972_20110509,3
ZSP0972_20110510,4


In [30]:
# save the file to a pickle
df_insiderWebCount.to_pickle('insiderWebCount.pkl')

In [31]:
# check - which cats have the hight suspicious counts - should there be a negation?
df_test[df_test['isInsiderWeb'] == 1]['domainCatsString'].value_counts()

['File Storage', 'Photo Sharing', 'Visual Search Engines', 'Photo Search and Images']                                   1071
['Ecommerce/Shopping', 'Forums/Message boards', 'Shopping']                                                              625
['Movies', 'Forums/Message boards', 'Entertainment']                                                                     486
['Blogs', 'Software/Technology', 'Forums/Message boards', 'Podcasts', 'Online Document Sharing and Collaboration']       235
['Jobs/Employment', 'Business Services', 'Business and Industry']                                                        233
['File Storage', 'File Transfer Services']                                                                               133
['Social Networking', 'Forums/Message boards', 'Online Communities']                                                     116
['File Storage', 'Search Engines', 'Software/Technology', 'Webmail', 'Search Engines and Portals']                        96


(raylet) [2023-04-01 14:23:48,889 E 774348 774348] (raylet) node_manager.cc:3097: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 6809ec1799f2218e377e09d93bca1b02d67260156df5bfa672826b4c, IP: 192.168.0.202) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.0.202`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) [2023-04-01 14:34:48,909 E 774348 774348] (raylet) node_manager.cc:3097: 1 Workers (tasks / actors) kill